In [1]:

# Import required libraries
import chess
import chess.engine
import chess.pgn
import subprocess
import os
import json
import csv
from IPython.display import display
from statistics import mean

# Paths to your engines and Maia models
stockfish_path = "/opt/homebrew/bin/stockfish"
maia_model_paths = {
    1100: "maia_weights/maia-1100.pb.gz",
    1200: "maia_weights/maia-1200.pb.gz",
    1300: "maia_weights/maia-1300.pb.gz",
    1400: "maia_weights/maia-1400.pb.gz",
    1500: "maia_weights/maia-1500.pb.gz",
    1600: "maia_weights/maia-1600.pb.gz",
    1700: "maia_weights/maia-1700.pb.gz",
    1800: "maia_weights/maia-1800.pb.gz",
    1900: "maia_weights/maia-1900.pb.gz",
}

# Initialize Stockfish engine
print("Initializing Stockfish engine...")
stockfish_engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

# Initialize Maia engines
print("Initializing Maia engines...")
maia_engines = {}
for elo, path in maia_model_paths.items():
    maia_engines[elo] = chess.engine.SimpleEngine.popen_uci(["lc0", f"--weights={path}"])

Initializing Stockfish engine...


<UciProtocol (pid=20359)>: stderr >>        _
<UciProtocol (pid=20359)>: stderr >> |   _ | |
<UciProtocol (pid=20359)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=20360)>: stderr >>        _
<UciProtocol (pid=20360)>: stderr >> |   _ | |
<UciProtocol (pid=20360)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=20361)>: stderr >>        _
<UciProtocol (pid=20361)>: stderr >> |   _ | |
<UciProtocol (pid=20361)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=20362)>: stderr >>        _
<UciProtocol (pid=20362)>: stderr >> |   _ | |
<UciProtocol (pid=20362)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=20363)>: stderr >>        _
<UciProtocol (pid=20363)>: stderr >> |   _ | |
<UciProtocol (pid=20363)>: stderr >> |_ |_ |_| v0.30.0+git.dirty built Jul 21 2023
<UciProtocol (pid=20364)>: stderr >>        _
<UciProtocol (pid=20364)>: stderr >> |   _ | |
<UciProtocol (pid=20364)>: 

Initializing Maia engines...


In [2]:
# Function to load games from a PGN file
def load_games_from_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    print(f"Loaded {len(games)} games from PGN file {file_path}.")
    return games

# Function to find classic puzzles in a game
def find_classic_puzzles(game, stockfish_engine, threshold=2.00):
    board = game.board()
    puzzles = []

    for move in game.mainline_moves():
        fen_before = board.fen()
        player = board.turn  # True for White, False for Black

        # Analyze the position before the move
        info = stockfish_engine.analyse(board, chess.engine.Limit(time=0.1), multipv=2)
        if len(info) < 2:
            continue

        best_move = info[0]["pv"][0]
        best_eval = info[0]["score"].relative.score(mate_score=10000)
        second_best_eval = info[1]["score"].relative.score(mate_score=10000)

        # Check if the difference is large enough
        if abs(best_eval - second_best_eval) >= threshold:
            board.push(best_move)
            if not board.is_capture(best_move):
                puzzles.append((fen_before, best_move, best_eval, second_best_eval))
                print(f"Puzzle found: {fen_before}, Best move: {best_move}, Eval difference: {abs(best_eval - second_best_eval)}")
            board.pop()

        board.push(move)

    return puzzles

# Function to get multiple best moves from Maia using multipv
def get_multiple_best_moves(engine, board, n=3):
    result = engine.analyse(board, chess.engine.Limit(nodes=1000), multipv=n)
    best_moves = [(info["pv"][0], info["score"].relative.score(mate_score=10000)) for info in result]
    # Sort the best moves based on their evaluation scores
    best_moves_sorted = sorted(best_moves, key=lambda x: x[1], reverse=True)
    return best_moves_sorted


In [3]:
# Function to evaluate Maia easiness
def evaluate_maia_easiness(fen, best_move, maia_engines, multipv=3):
    board = chess.Board(fen)
    easiness_scores = {}
    print(f"Evaluating Maia easiness for FEN: {fen}, Best Move: {best_move}")

    for elo, engine in maia_engines.items():
        multiple_best_moves = get_multiple_best_moves(engine, board, multipv)
        print(f"Maia {elo} best moves: {multiple_best_moves}")

        move_probabilities = {move: (i + 1) / len(multiple_best_moves) for i, (move, score) in enumerate(multiple_best_moves)}

        if best_move in move_probabilities:
            easiness_score = move_probabilities[best_move]
        else:
            easiness_score = 1.0  # Assign 1.0 if the best move is not in the PV list

        easiness_scores[elo] = easiness_score

    # Combine easiness scores from different models into a single score
    combined_easiness_score = mean(easiness_scores.values()) if easiness_scores else 1.0
    return combined_easiness_score


In [4]:
# Function to save results to CSV
def save_results_to_csv(csv_file_path, data):
    with open(csv_file_path, "a", newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        for fen, best_move, best_eval, second_best_eval, maia_easiness in data:
            csvwriter.writerow([
                fen,
                best_move.uci(),  # Convert Move object to UCI notation
                best_eval,
                second_best_eval,
                maia_easiness
            ])

# Function to save results to JSON
def save_results_to_json(json_file_path, data):
    if os.path.exists(json_file_path):
        with open(json_file_path, "r") as jsonfile:
            json_data = json.load(jsonfile)
    else:
        json_data = []

    json_data.extend([{
        "fen": fen,
        "best_move": best_move.uci(),  # Convert Move object to UCI notation
        "best_eval": best_eval,
        "second_best_eval": second_best_eval,
        "maia_easiness": maia_easiness
    } for fen, best_move, best_eval, second_best_eval, maia_easiness in data])

    with open(json_file_path, "w") as jsonfile:
        json.dump(json_data, jsonfile, indent=4)

# Paths to save results
csv_file_path = "classic_puzzles.csv"
json_file_path = "classic_puzzles.json"

# Initialize CSV file with headers
with open(csv_file_path, "w", newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["FEN", "Best Move", "Best Eval", "Second Best Eval", "Maia Easiness"])

# Load PGN files and find classic puzzles
pgn_directory = "analysis_pgns_test"
all_puzzles = []

print("Starting to find classic puzzles in PGN files...")

for pgn_file in os.listdir(pgn_directory):
    if pgn_file.endswith(".pgn"):
        pgn_file_path = os.path.join(pgn_directory, pgn_file)
        print(f"Analyzing {pgn_file_path}...")
        all_games = load_games_from_pgn(pgn_file_path)

        for game in all_games:
            print(f"Analyzing game {game.headers.get('Event', 'Unknown Event')}")
            puzzles = find_classic_puzzles(game, stockfish_engine)
            for fen, best_move, best_eval, second_best_eval in puzzles:
                maia_easiness = evaluate_maia_easiness(fen, best_move, maia_engines)
                all_puzzles.append((fen, best_move, best_eval, second_best_eval, maia_easiness))

            # Save results incrementally
            save_results_to_csv(csv_file_path, puzzles)
            save_results_to_json(json_file_path, puzzles)

print("Classic puzzles analysis complete. Results saved.")


Starting to find classic puzzles in PGN files...
Analyzing analysis_pgns_test/4c6eS6SH---1969 Petrosian vs. Spassky .pgn...
Loaded 4 games from PGN file analysis_pgns_test/4c6eS6SH---1969 Petrosian vs. Spassky .pgn.
Analyzing game FIDE World Championship Match 1969
Analyzing game FIDE World Championship Match 1969
Analyzing game FIDE World Championship Match 1969
Analyzing game FIDE World Championship Match 1969
Analyzing analysis_pgns_test/4S5UuAGn---1986 Karpov vs. Kasparov III.pgn...
Loaded 1 games from PGN file analysis_pgns_test/4S5UuAGn---1986 Karpov vs. Kasparov III.pgn.
Analyzing game FIDE World Championship Match 1986
Analyzing analysis_pgns_test/TFFyOff1---1908 Lasker vs. Tarrasch.pgn...
Loaded 7 games from PGN file analysis_pgns_test/TFFyOff1---1908 Lasker vs. Tarrasch.pgn.
Analyzing game World Championship Match 1908
Analyzing game World Championship Match 1908
Analyzing game World Championship Match 1908
Analyzing game World Championship Match 1908
Analyzing game World Cha

In [5]:
# Cleanup Maia engines
print("Cleaning up engines...")
for engine in maia_engines.values():
    engine.quit()
stockfish_engine.quit()
print("Engines cleaned.")


Cleaning up engines...
Engines cleaned.
